In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.nn.functional as F

import os
import numpy as np

import models
import torchvision
import torchvision.transforms as transforms
import utils
import shutil

from bisect import bisect_right
import time
import math

In [17]:
data_path = './data'
checkpoint_dir = './checkpoint'
dataset = 'cifar100'
arch = 'wrn_16_2'             
init_lr = 0.05
weight_decay = 5e-4
lr_type = 'multistep'
milestones = [150, 180, 210]
sgdr_t = 300
warmup_epoch = 0
epochs = 240
batch_size = 64
num_workers = 8
resume = False
evaluate = False
gpu_id = '0'
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id
manual_seed = 0
kd_T = 3
num_classes = 100

In [18]:
log_txt = 'result/'+ str(os.path.basename('kushagrabhushan/TrAIL').split('.')[0]) + '_'+\
          'arch' + '_' +  arch + '_'+\
          'dataset' + '_' +  dataset + '_'+\
          'seed'+ str(manual_seed) +'.txt'


log_dir = str(os.path.basename('kushagrabhushan/TrAIL').split('.')[0]) + '_'+\
          'arch'+ '_' + arch + '_'+\
          'dataset' + '_' +  dataset + '_'+\
          'seed'+ str(manual_seed)
checkpoint_dir = os.path.join(checkpoint_dir, log_dir)
if not os.path.isdir(checkpoint_dir):
    os.makedirs(checkpoint_dir)

if resume is False and evaluate is False:
    with open(log_txt, 'a+') as f:
        f.seek(0)
        f.truncate()


In [19]:
DEFAULT_SEED = 0
np.random.seed(DEFAULT_SEED)
torch.manual_seed(DEFAULT_SEED)
torch.cuda.manual_seed_all(DEFAULT_SEED)
torch.set_printoptions(precision=4)

In [20]:
model = getattr(models, arch)
net = model(num_classes=num_classes).cuda()
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

In [21]:
trainset = torchvision.datasets.CIFAR100(root=data_path, train=True, download=True,
                                        transform=transforms.Compose([
                                            transforms.RandomCrop(32, padding=4),
                                            transforms.RandomHorizontalFlip(),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.5071, 0.4867, 0.4408],
                                                                [0.2675, 0.2565, 0.2761])
                                        ]))

testset = torchvision.datasets.CIFAR100(root=data_path, train=False, download=True,
                                        transform=transforms.Compose([
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.5071, 0.4867, 0.4408],
                                                                [0.2675, 0.2565, 0.2761]),
                                        ]))
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True,
                                    pin_memory=(torch.cuda.is_available()))

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False,
                                    pin_memory=(torch.cuda.is_available()))


Files already downloaded and verified
Files already downloaded and verified


In [22]:
def correct_num(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    correct = pred.eq(target.view(-1, 1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:, :k].float().sum()
        res.append(correct_k)
    return res

In [23]:
def adjust_lr(optimizer, epoch, args, step=0, all_iters_per_epoch=0, eta_min=0.):
    cur_lr = 0.
    if epoch < args['warmup_epoch']:
        cur_lr = args['init_lr'] * float(1 + step + epoch*all_iters_per_epoch)/(args['warmup_epoch'] *all_iters_per_epoch)
    else:
        epoch = epoch - args['warmup_epoch']
        if args['lr_type'] == 'SGDR':
            i = int(math.log2(epoch / args['sgdr_t'] + 1))
            T_cur = epoch - args['sgdr_t'] * (2 ** (i) - 1)
            T_i = (args['sgdr_t'] * 2 ** i)

            cur_lr = eta_min + 0.5 * (args['init_lr'] - eta_min) * (1 + np.cos(np.pi * T_cur / T_i))

        elif args['lr_type'] == 'multistep':
            cur_lr = args['init_lr'] * 0.1 ** bisect_right(args['milestones'], epoch)

    for param_group in optimizer.param_groups:
        param_group['lr'] = cur_lr
    return cur_lr

In [24]:
class DistillKL(nn.Module):
    """Distilling the Knowledge in a Neural Network"""
    def __init__(self, T):
        super(DistillKL, self).__init__()
        self.T = T

    def forward(self, y_s, y_t):
        p_s = F.log_softmax(y_s/self.T, dim=1)
        p_t = F.softmax(y_t/self.T, dim=1)
        loss = F.kl_div(p_s, p_t, reduction='batchmean') * (self.T**2)
        return loss

In [25]:
def train(epoch, criterion_list, optimizer):
    train_loss = 0.
    train_loss_cls = 0.
    train_loss_div = 0.
    train_loss_kd = 0.

    top1_num = 0
    top5_num = 0
    total = 0

    if epoch >= warmup_epoch:
        lr = adjust_lr(optimizer, epoch, args)

    start_time = time.time()
    criterion_cls = criterion_list[0]
    criterion_div = criterion_list[1]

    net.train()
    for batch_idx, (input, target) in enumerate(trainloader):
        batch_start_time = time.time()
        input = input.float().cuda()
        target = target.cuda()

        if epoch < warmup_epoch:
            lr = adjust_lr(optimizer, epoch, args, batch_idx, len(trainloader))

        optimizer.zero_grad()
        logits = net(input)

        loss_cls = criterion_cls(logits, target)

        loss = loss_cls
        loss.backward()
        optimizer.step()

        train_loss += loss.item() / len(trainloader)
        train_loss_cls += loss_cls.item() / len(trainloader)

        top1, top5 = correct_num(logits, target, topk=(1, 5))
        top1_num += top1
        top5_num += top5
        total += target.size(0)

        print('Epoch:{}, batch_idx:{}/{}, lr:{:.5f}, Duration:{:.2f}, Top-1 Acc:{:.4f}'.format(
            epoch, batch_idx, len(trainloader), lr, time.time()-batch_start_time, (top1_num/(total)).item()))


    acc1 = round((top1_num/total).item(), 4)
    acc5 = round((top5_num/total).item(), 4)



    with open(log_txt, 'a+') as f:
        f.write('Epoch:{}\t lr:{:.4f}\t duration:{:.3f}'
                '\n train_loss:{:.5f}\t train_loss_cls:{:.5f}'
                '\nTop-1 accuracy: {} \n'
                .format(epoch, lr, time.time() - start_time,
                        train_loss, train_loss_cls,
                        str(acc1)))
        if dataset == 'imagenet':
            f.write('Top-5 accuracy:{}\n'.format(str(acc5)))

In [26]:
def test(epoch, criterion_cls):
    net.eval()
    global best_acc
    test_loss_cls = 0.

    top1_num = 0
    top5_num = 0
    total = 0
    

    with torch.no_grad():
        for batch_idx, (inputs, target) in enumerate(testloader):
            batch_start_time = time.time()
            inputs, target = inputs.cuda(), target.cuda()
            logits = net(inputs)

            loss_cls = criterion_cls(logits, target)

            test_loss_cls += loss_cls.item()/ len(testloader)

            top1, top5 = correct_num(logits, target, topk=(1, 5))
            top1_num += top1
            top5_num += top5
            total += target.size(0)

            print('Epoch:{}, batch_idx:{}/{}, Duration:{:.2f}, Top-1 Acc:{:.4f}'.format(
                epoch, batch_idx, len(testloader), time.time()-batch_start_time, (top1_num/(total)).item()))

        acc1 = round((top1_num/total).item(), 4)
        acc5 = round((top5_num/total).item(), 4)

        with open(log_txt, 'a+') as f:
            f.write('test epoch:{}\t test_loss_cls:{:.5f}\t Top-1 accuracy:{}\n'
                    .format(epoch, test_loss_cls, str(acc1)))
            if dataset == 'imagenet':
                f.write('Top-5 accuracy:{}\n'.format(str(acc5)))
        print('test epoch:{}\t accuracy:{}\n'.format(epoch, str(acc1)))

    return acc1

In [27]:
best_acc = 0.
start_epoch = 0  
criterion_cls = nn.CrossEntropyLoss()
#criterion_div = utils.DistillKL(kd_T)

In [29]:
best_acc = 0.  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
criterion_cls = nn.CrossEntropyLoss()
criterion_div = DistillKL(kd_T)

if evaluate: 
    print('load pre-trained weights from: {}'.format(os.path.join(checkpoint_dir, str(model.__name__) + '_best.pth.tar')))     
    checkpoint = torch.load(os.path.join(checkpoint_dir, str(model.__name__) + '_best.pth.tar'),
                            map_location=torch.device('cpu'))
    net.module.load_state_dict(checkpoint['net'])
    best_acc = checkpoint['acc']
    start_epoch = checkpoint['epoch'] + 1
    test(start_epoch, criterion_cls)
else:
    trainable_list = nn.ModuleList([])
    trainable_list.append(net)
    optimizer = optim.SGD(trainable_list.parameters(),
                          lr=0.1, momentum=0.9, weight_decay=weight_decay, nesterov=True)

    criterion_list = nn.ModuleList([])
    criterion_list.append(criterion_cls)  # classification loss
    criterion_list.append(criterion_div)  # KL divergence loss, original knowledge distillation
    criterion_list.cuda()

    if resume:
        print('load pre-trained weights from: {}'.format(os.path.join(checkpoint_dir, str(model.__name__) + '.pth.tar')))
        checkpoint = torch.load(os.path.join(checkpoint_dir, str(model.__name__) + '.pth.tar'),
                                map_location=torch.device('cpu'))
        net.module.load_state_dict(checkpoint['net'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        best_acc = checkpoint['acc']
        start_epoch = checkpoint['epoch'] + 1

    for epoch in range(start_epoch, epochs):
        train(epoch, criterion_list, optimizer)
        acc = test(epoch, criterion_cls)

        state = {
            'net': net.module.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'optimizer': optimizer.state_dict()
        }
        torch.save(state, os.path.join(checkpoint_dir, str(model.__name__) + '.pth.tar'))

        is_best = False
        if best_acc < acc:
            best_acc = acc
            is_best = True

        if is_best:
            shutil.copyfile(os.path.join(checkpoint_dir, str(model.__name__) + '.pth.tar'),
                            os.path.join(checkpoint_dir, str(model.__name__) + '_best.pth.tar'))

    print('Evaluate the best model:')
    print('load pre-trained weights from: {}'.format(os.path.join(checkpoint_dir, str(model.__name__) + '_best.pth.tar')))
    evaluate = True
    checkpoint = torch.load(os.path.join(checkpoint_dir, str(model.__name__) + '_best.pth.tar'),
                            map_location=torch.device('cpu'))
    net.module.load_state_dict(checkpoint['net'])
    start_epoch = checkpoint['epoch']
    top1_acc = test(start_epoch, criterion_cls)

    with open(log_txt, 'a+') as f:
        f.write('best_accuracy: {} \n'.format(best_acc))
    print('best_accuracy: {} \n'.format(best_acc))
    os.system('cp ' + log_txt + ' ' + checkpoint_dir)

Epoch:237, batch_idx:0/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9375
Epoch:237, batch_idx:1/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9531
Epoch:237, batch_idx:2/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9531
Epoch:237, batch_idx:3/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9609
Epoch:237, batch_idx:4/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9625
Epoch:237, batch_idx:5/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9557
Epoch:237, batch_idx:6/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9554
Epoch:237, batch_idx:7/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9570
Epoch:237, batch_idx:8/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9549
Epoch:237, batch_idx:9/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9531
Epoch:237, batch_idx:10/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9503
Epoch:237, batch_idx:11/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9518
Epoch:237, batch_idx:12/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9483
Epoch:237, batch_idx:13/782, lr:0.00005, Duration:0.02, Top-1

Epoch:237, batch_idx:114/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9549
Epoch:237, batch_idx:115/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9553
Epoch:237, batch_idx:116/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9555
Epoch:237, batch_idx:117/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9554
Epoch:237, batch_idx:118/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9548
Epoch:237, batch_idx:119/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9548
Epoch:237, batch_idx:120/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9549
Epoch:237, batch_idx:121/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9549
Epoch:237, batch_idx:122/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9546
Epoch:237, batch_idx:123/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9546
Epoch:237, batch_idx:124/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9549
Epoch:237, batch_idx:125/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:237, batch_idx:126/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9548
Epoch:237, batch_idx:127/782, lr:0.000

Epoch:237, batch_idx:227/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9540
Epoch:237, batch_idx:228/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:237, batch_idx:229/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9539
Epoch:237, batch_idx:230/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9538
Epoch:237, batch_idx:231/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9537
Epoch:237, batch_idx:232/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9538
Epoch:237, batch_idx:233/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9536
Epoch:237, batch_idx:234/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9535
Epoch:237, batch_idx:235/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9537
Epoch:237, batch_idx:236/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9539
Epoch:237, batch_idx:237/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9536
Epoch:237, batch_idx:238/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9535
Epoch:237, batch_idx:239/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9535
Epoch:237, batch_idx:240/782, lr:0.000

Epoch:237, batch_idx:340/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:341/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:237, batch_idx:342/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:343/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:344/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:345/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9542
Epoch:237, batch_idx:346/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9543
Epoch:237, batch_idx:347/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:348/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:349/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:350/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:237, batch_idx:351/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:237, batch_idx:352/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:237, batch_idx:353/782, lr:0.000

Epoch:237, batch_idx:453/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9541
Epoch:237, batch_idx:454/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:455/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9543
Epoch:237, batch_idx:456/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:457/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:237, batch_idx:458/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9542
Epoch:237, batch_idx:459/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9542
Epoch:237, batch_idx:460/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:461/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:462/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9542
Epoch:237, batch_idx:463/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9542
Epoch:237, batch_idx:464/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:465/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:237, batch_idx:466/782, lr:0.000

...

Epoch:239, batch_idx:643/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:644/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:645/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:646/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:647/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:648/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9547
Epoch:239, batch_idx:649/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9546
Epoch:239, batch_idx:650/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9546
Epoch:239, batch_idx:651/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9545
Epoch:239, batch_idx:652/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9545
Epoch:239, batch_idx:653/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9545
Epoch:239, batch_idx:654/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9545
Epoch:239, batch_idx:655/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9544
Epoch:239, batch_idx:656/782, lr:0.000

Epoch:239, batch_idx:754/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9543
Epoch:239, batch_idx:755/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:756/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:757/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:758/782, lr:0.00005, Duration:0.04, Top-1 Acc:0.9542
Epoch:239, batch_idx:759/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:760/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:761/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9542
Epoch:239, batch_idx:762/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:239, batch_idx:763/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9541
Epoch:239, batch_idx:764/782, lr:0.00005, Duration:0.03, Top-1 Acc:0.9540
Epoch:239, batch_idx:765/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:239, batch_idx:766/782, lr:0.00005, Duration:0.02, Top-1 Acc:0.9541
Epoch:239, batch_idx:767/782, lr:0.000

Epoch:239, batch_idx:101/157, Duration:0.01, Top-1 Acc:0.7350
Epoch:239, batch_idx:102/157, Duration:0.01, Top-1 Acc:0.7336
Epoch:239, batch_idx:103/157, Duration:0.01, Top-1 Acc:0.7336
Epoch:239, batch_idx:104/157, Duration:0.01, Top-1 Acc:0.7341
Epoch:239, batch_idx:105/157, Duration:0.01, Top-1 Acc:0.7339
Epoch:239, batch_idx:106/157, Duration:0.01, Top-1 Acc:0.7341
Epoch:239, batch_idx:107/157, Duration:0.01, Top-1 Acc:0.7348
Epoch:239, batch_idx:108/157, Duration:0.01, Top-1 Acc:0.7355
Epoch:239, batch_idx:109/157, Duration:0.01, Top-1 Acc:0.7352
Epoch:239, batch_idx:110/157, Duration:0.01, Top-1 Acc:0.7351
Epoch:239, batch_idx:111/157, Duration:0.01, Top-1 Acc:0.7351
Epoch:239, batch_idx:112/157, Duration:0.01, Top-1 Acc:0.7355
Epoch:239, batch_idx:113/157, Duration:0.01, Top-1 Acc:0.7360
Epoch:239, batch_idx:114/157, Duration:0.01, Top-1 Acc:0.7357
Epoch:239, batch_idx:115/157, Duration:0.01, Top-1 Acc:0.7356
Epoch:239, batch_idx:116/157, Duration:0.01, Top-1 Acc:0.7356
Epoch:23

Epoch:237, batch_idx:78/157, Duration:0.01, Top-1 Acc:0.7366
Epoch:237, batch_idx:79/157, Duration:0.01, Top-1 Acc:0.7363
Epoch:237, batch_idx:80/157, Duration:0.01, Top-1 Acc:0.7373
Epoch:237, batch_idx:81/157, Duration:0.01, Top-1 Acc:0.7382
Epoch:237, batch_idx:82/157, Duration:0.01, Top-1 Acc:0.7378
Epoch:237, batch_idx:83/157, Duration:0.01, Top-1 Acc:0.7370
Epoch:237, batch_idx:84/157, Duration:0.01, Top-1 Acc:0.7366
Epoch:237, batch_idx:85/157, Duration:0.01, Top-1 Acc:0.7360
Epoch:237, batch_idx:86/157, Duration:0.01, Top-1 Acc:0.7364
Epoch:237, batch_idx:87/157, Duration:0.01, Top-1 Acc:0.7381
Epoch:237, batch_idx:88/157, Duration:0.01, Top-1 Acc:0.7386
Epoch:237, batch_idx:89/157, Duration:0.01, Top-1 Acc:0.7392
Epoch:237, batch_idx:90/157, Duration:0.01, Top-1 Acc:0.7406
Epoch:237, batch_idx:91/157, Duration:0.01, Top-1 Acc:0.7395
Epoch:237, batch_idx:92/157, Duration:0.01, Top-1 Acc:0.7408
Epoch:237, batch_idx:93/157, Duration:0.01, Top-1 Acc:0.7399
Epoch:237, batch_idx:94/

0